<a href="https://colab.research.google.com/github/seanlam74/GMS6812/blob/master/GMS6812_PredictionDemo22Sep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GMS6812 Predictive Modeling Demo for Healthcare Analytics

#### Instructor: Sean Lam (Email: lam.shao.wei@singhealth.com.sg), Health Services Research, SingHealth and Duke NUS

In this demo, you will learn the following,
- Performed Explanatory Data Analysis and applied extensive feature engineering, feature selection and on Diabetes patient’s hospital readmission data.
- Built Decision Tree and Logistic classifiers in python using Scikit-Learn to predict which patients might be readmitted to the hospital.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from scipy.stats import chisquare, ks_2samp
import numpy as np

# 1. Data Loading

In [ ]:
# Reading the data into python with "pandas", file in GItHub
url = 'https://github.com/seanlam74/GMS6812/blob/master/training_data.csv'
df = pd.read_csv(url, error_bad_lines=False)
data.shape

In [ ]:
# Reading the data into python with "pandas"
data=pd.read_csv('training_data.csv',header=0,skiprows=0,engine='python')
data.shape

In [ ]:
# printing out first 5 observations
data.head()

In [ ]:
#Printing all the columns in the dataset
print(data.info())

In [ ]:
#Descriptive statistics of all the rows
pd.set_option('display.expand_frame_repr', False)
print(data.describe())

# 2. Data Wrangling

## 2.1 Data Quality Checks

Before we do analysis we need to make sure that there are no duplicate rows.<br> 
By looking at the data we can confirm that encounter_id is unique and we are checking if there are any duplicate encounter_ids and we find that there aren't any.

In [ ]:
ids = data['encounter_id']
data[ids.duplicated()]

In [ ]:
#checking if there are any duplicates in encounter_id - no duplicates found
ids = data['encounter_id']
data[ids.isin(ids[ids.duplicated()])]

In [ ]:
# remove duplicates and check again
data = data.drop_duplicates()
data[ids.duplicated()]

In [ ]:
# Considering values with '?' as missing values.
data = data.replace('?', np.NaN )

In [ ]:
# Two observation is unknown in gender. Replace it as a null value
print('gender', data['gender'][data['gender'] == 'Unknown/Invalid'].count())
data = data.replace('Unknown/Invalid', np.NaN )

In [ ]:
# finding the number of null values in each column
print(data.isnull().sum())

Any row that has more than 40% missing values is rejected.

In [ ]:
# Weight has almost 99% of missing data ; Payer_code and medical_speciality has around 45% of missing data. 
data.drop(['weight','payer_code','medical_specialty','diag_2', 'diag_3'],axis=1,inplace=True)
data.shape

In [ ]:
#Dropping rows with missing values
data.dropna(inplace=True)
data.shape

In [ ]:
data[0:3]

In [ ]:
# creating a list of categorical and numeric columns names
categorical=data.select_dtypes(include=['object']) # select strings
numeric=data.select_dtypes(exclude=['object'])
print(categorical.columns.values)
print(numeric.columns.values)

In [ ]:
# printing the frequency count of all the categorical features
for col in categorical:
    print(categorical[col].value_counts())

In [ ]:
# printing the frequency of all the numeric features
for col in numeric:
    print(numeric[col].value_counts())
# Didn't find any outliers

In [ ]:
for col in categorical:
    categorical[col].value_counts().plot(kind='bar')
    plt.show()    

In [ ]:
#data['max_glu_serum'].unique()

In [ ]:
for col in numeric:
    numeric[col].plot.hist(bins=6)
    plt.show()

In [ ]:
# Deleting the 2 columns because they have all the observations only in one category 
data.drop(['examide','citoglipton'],axis=1,inplace=True) # axis=1 column
data.shape

In [ ]:
#Making the target variable and other variables binary
data = data[data['readmitted'].isin(['Y', 'N'])] # keep those with Y/N
data['readmitted'] = data['readmitted'].apply(lambda x: 0 if x == "N" else 1)
data['change'] = data['change'].apply(lambda x: 0 if x == "No" else 1)
data['gender'] = data['gender'].apply(lambda x: 0 if x == "Female" else 1)
data['diabetesMed'] = data['diabetesMed'].apply(lambda x: 0 if x == "No" else 1)
data.shape

In [ ]:
# There are 3 types of visits to a hospital. 1) Inpatient 2) Outpatient 3) Emergency.
# Combining them into a single column
data['total_visits'] = data['number_outpatient'] + data['number_emergency'] + data['number_inpatient']

We have age feature which is given in bins. We have to changed it with the average value. eg: for age 0-10 we took the average age which is 5.

In [ ]:
# defining a function to give average value for the age
def agecategory(x):
    
    if x == "[0-10)" :
        return 5
    elif x == "[10-20)":
        return 15
    elif x == "[20-30)":
        return 25
    elif x == "[30-40)":
        return 35
    elif x == "[40-50)":
        return 45
    elif x == "[50-60)":
        return 55
    elif x == "[60-70)":
        return 65
    elif x == "[70-80)":
        return 75
    else:
        return 0        

In [ ]:
# replacing the age bins with their average value
data['age'] = data['age'].apply(lambda x: agecategory(x))

In [ ]:
data['age'][:10]

There are 23 treatments of which 2 treatments are never used by patients and we took the number of treatments the patient has undergone as a feature which will be used for analysis.

In [ ]:
# There are many treatments from which a doctor would recommend the patient, lets combine all the treatments into one dataframe
treatments = ['metformin' ,'repaglinide','nateglinide','chlorpropamide','glimepiride','acetohexamide' ,'glipizide',\
              'glyburide', 'tolbutamide', 'pioglitazone','rosiglitazone', 'acarbose' ,'miglitol' ,'troglitazone', \
              'tolazamide', 'insulin' ,'glyburide-metformin','glipizide-metformin', \
              'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone']

In [ ]:
print(treatments)

In [ ]:
# Assigning a value of 0 if there are not undergoing treatment and assigning 1 even if they are taking\
# increasing/decreasing/steady dosage
for i in treatments:
    data[i] = data[i].apply(lambda x: 0 if x == "No" else 1)

In [ ]:
# finding out total number of treatments taken by patient
data['treatments_taken'] = np.zeros((len(data['metformin']))) # create zero-vector of all rows
for col in treatments:
    data['treatments_taken'] += data[col]

In [ ]:
data.head()

In [ ]:
# A1C > 6.4 implies that the patient has diabetes. Therefore, considered values greater than 7 and 8 together.
# other 2 categories : Norm and None ; Norm implies the values in the normal range ; None implies no test conducted;
data['A1Cresult'] = data['A1Cresult'].apply(lambda x: 0 if x == "None" else (1 if x=="Norm" else 2) )
data['max_glu_serum'] = data['max_glu_serum'].apply(lambda x: 0 if x == "None" else (1 if x=="Norm" else 2) )

In [ ]:
#Based on information in https://www.hindawi.com/journals/bmri/2014/781670/tab2/.Classified diagnosis into 9 categories
#Categories[0-8]: Other,Circulatory, Respiratory,Digestive, Diabetes,Injury, Musculoskeletal,Genitourinary,Neoplasms
#defining the function to classify the numbers into one of the 8 categories

def getCategor(x):
    if 'V' in str(x) or 'E' in str(x):
        return 0
    
    x = float(x)
    
    if (x >= 390 and x <= 459) or np.floor(x) == 785:
        return 1
    elif (x >= 460 and x <= 519) or np.floor(x) == 786:
        return 2
    elif (x >= 520 and x <= 579) or np.floor(x) == 787:
        return 3
    elif np.floor(x) == 250:
        return 4
    elif x >= 800 and x <= 999:
        return 5
    elif x >= 710 and x <= 739:
        return 6
    elif (x >= 580 and x <= 629) or np.floor(x) == 788:
        return 7
    elif x >= 140 and x <= 239:
        return 8
    else:
        return 0
        

In [ ]:
#changing the values into categories
data['diag_1_category'] = data['diag_1'].apply(lambda x: getCategor(x))

In [ ]:
data['diag_1_category'][:10]

In [ ]:
list(data)

Some patients in the data have more than one encounters, we need to make sure to remove the multiple patient visits because that might cause bias in our predictions. For that reason we remove all the visits by a patient other than their first visit (i.e., index visit).

In [ ]:
# Check for readmitted patients and remove all visits other than the 1st visit
#patients = data['patient_nbr']
#data[patients.isin(patients[patients.duplicated()])]

In [ ]:
#dropping the patients encounters other than 1st visit
data = data.drop_duplicates(subset= ['patient_nbr'], keep = 'first')
data.shape

Variables like admission_type_id, discharge_despotion_id etc does not have any intrinsic value associated with them. So we make them categorical variables.

In [ ]:
# coercing the admission_type_id, discharge_disposition_id, admission_source_id diag_1_category, \
# max_glu_serum, A1Cresult into categorical since the magnitudes does not have any intrinsic value
data['admission_type_id'] = data['admission_type_id'].astype('object')
data['admission_source_id'] = data['admission_source_id'].astype('object')
data['discharge_disposition_id'] = data['discharge_disposition_id'].astype('object')
data['diag_1_category'] = data['diag_1_category'].astype('object')
data['max_glu_serum'] = data['max_glu_serum'].astype('object')
data['A1Cresult'] = data['A1Cresult'].astype('object')

Features like encounter_id, patient_nbr are for identity purpose and do not contribute towards predictions, so we remove them from analysis. Variables like number_outpatient, number_emergency are being used to create new variable which are being used for analysis. So we can remove other unnecessary variables.

In [ ]:
#list(data)

In [ ]:
# creating a list for unnecessary columns
delete_columns = ['encounter_id','patient_nbr','number_outpatient','number_emergency','number_inpatient',\
                 'metformin','repaglinide','nateglinide','chlorpropamide','glimepiride','acetohexamide','glipizide'\
                  ,'glyburide','tolbutamide','pioglitazone','rosiglitazone','acarbose','miglitol','troglitazone' \
                  ,'tolazamide','insulin','glyburide-metformin','glipizide-metformin','glimepiride-pioglitazone',\
                  'metformin-rosiglitazone','metformin-pioglitazone','diag_1']

In [ ]:
#dropping the unnecessary columns
data.drop(delete_columns, inplace=True, axis=1)

In [ ]:
data.head()

In [ ]:
#import numpy as np
#import seaborn as sns
#%matplotlib inline
from scipy.stats import kurtosis
from scipy.stats import skew
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#creating a list of categorical and numeric lists
categorical=data.select_dtypes(include=['object'])
numeric=data.select_dtypes(exclude=['object'])
print(categorical.columns.values)
print(numeric.columns.values)

In [ ]:
# creating dummies for all the categorical variables and deleting the original columns
nominal_columns = ['race', 'admission_type_id', 'discharge_disposition_id','admission_source_id' ,'diag_1_category'\
                  , 'max_glu_serum', 'A1Cresult']
dummy_df = pd.get_dummies(data[nominal_columns])
data = pd.concat([data, dummy_df], axis=1)
data = data.drop(nominal_columns, axis=1)

In [ ]:
#list(data)

In [ ]:
data.head()

## Prepare training/test datasets

In [ ]:
data1 = data

In [ ]:
data1

In [ ]:
y = data1['readmitted']

In [ ]:
X = data1.drop('readmitted',axis=1)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.20, random_state=42)

## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score, precision_score, recall_score

Precision (also called positive predictive value) \
Recall (also known as sensitivity) \
F1 Score = 2 * (Precision * Recall) / (Precision + Recall)

In [ ]:
#from sklearn.model_selection import cross_val_score

#performance = []
#for max_depth in [2,3,5,7,10]:
#    dTree = DecisionTreeClassifier(criterion='entropy', class_weight = "balanced", max_depth=max_depth)
#    performance.append((max_depth, np.mean(cross_val_score(dTree, X_train, Y_train, cv = 10, scoring = "f1_micro"))))

In [ ]:
#print(performance)
#print("The best tree size is: ") 
#str(sorted(performance, key = lambda x: x[1])[-1][0])

In [ ]:
#X_train

In [ ]:
from sklearn.model_selection import cross_val_score, KFold

dTree = DecisionTreeClassifier(criterion='entropy', class_weight='balanced', max_depth = 5)
#kf = KFold(n_splits=10, shuffle=True, random_state=0)
dTree.fit(X_train, Y_train)

print("Test Results:")
y_prediction = dTree.predict(X_test)
print(classification_report(Y_test, y_prediction))

Note that in binary classification, recall of the positive class is also known as “sensitivity”; recall of the negative class is “specificity”.
\
Note: Cut-off values determine sensitivity, specificity, etc.

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() # Scaled data has zero mean and unit variance
X_train_normal = scaler.fit_transform(X_train)
X_test_normal = scaler.transform(X_test)

#model = LogisticRegressionCV(Cs = 10, cv = 10, class_weight = "balanced")
model = LogisticRegression(class_weight = "balanced")
model.fit(X_train_normal, Y_train)

In [ ]:
y_prediction = model.predict(X_test_normal)
print(classification_report(y_prediction, Y_test))

## Homework (unofficial)

### 1) To explore result reporting with the predicted probabilities
- ROC analysis, to report AUC value, sensitivity, specificity, positive predictive value, negative predictive value
- Plot precision-recall curve, etc

### 2) To build model with training dataset and validate it with (separate) test dataset
- Use the "test_data.csv"

### 3) To try different prediction methods
- Random forest
- XGBoost


## Reference
https://github.com/andrewwlong/diabetes_readmission